In [1]:
import tweepy
import pandas as pd
import sys
sys.path.append('../../')
import keys as k

In [2]:
# Tokens de acceso
consumer_key = k.API_KEY
consumer_secret = k.API_KEY_SECRET
access_token = k.Acces_Token
access_token_secret = k.Acces_Token_Secret

In [3]:
# Autenticación en la API de Twitter
auth = tweepy.OAuth1UserHandler(consumer_key, consumer_secret, access_token, access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit = True)

In [4]:
# Define las palabras clave relevantes para buscar los tweets
palabras_clave = ["bitcoin", "BTC", "$BTC"]

In [5]:
relevant_users_dict = {
    'user_id': [1, 2],
    'screen_name': ['@elonmusk', '@CoinDesk'],
    'relevant_nivel': [5, 4]
}

In [6]:
df_relevant_users = pd.DataFrame(relevant_users_dict)
df_relevant_users

,user_id,screen_name,relevant_nivel
0,1,@elonmusk,5
1,2,@CoinDesk,4


In [7]:
list_relevant_users = df_relevant_users['screen_name'].tolist()
list_relevant_users

['@elonmusk', '@CoinDesk']

In [8]:
# Obtén tweets basados en las palabras clave
tweets = []
for palabra_clave in palabras_clave:
    for tweet in tweepy.Cursor(api.search_tweets, palabra_clave, count=10, tweet_mode='extended').items(100):
        tweets.append(tweet)

Unauthorized: 401 Unauthorized
32 - Could not authenticate you.

In [ ]:
# Almacena los tweets en un DataFrame de Pandas con las columnas solicitadas
tweets_df = pd.DataFrame(columns=['tweet_id', 'timestamp', 'user_id', 'screen_name', 'full_name', 'text', 'hashtags', 'mentions', 'retweet_count', 'retweet_user', 'favorite_count', 'country', 'relevant', 'tweet_sentiment'])

for tweet in tweets:
    if not tweet.full_text.startswith('RT'):
        tweet_id = tweet.id
        timestamp = tweet.created_at
        user_id = tweet.user.id_str
        screen_name = tweet.user.screen_name
        full_name = tweet.user.name
        text = tweet.full_text
        hashtags = [tag['text'] for tag in tweet.entities['hashtags']]
        mentions = [mention['screen_name'] for mention in tweet.entities['user_mentions']]
        retweet_count = tweet.retweet_count
        retweet_user = []
        if retweet_count > 0:
            try:
                retweets = api.get_retweets(tweet_id, count=100)
                retweet_user = [retweet.user.screen_name for retweet in retweets]
            except tweepy.TweepyException as e:
                if e.response.status_code == 404:
                    # Retweet no disponible (error 404)
                    continue  # Continuar con el siguiente tweet
                else:
                    # Otro tipo de error
                    print('Error:', e)
                    break  # Salir del bucle en caso de otros errores
        favorite_count = tweet.favorite_count

        # Get the country of the tweet (if available)
        if tweet.place is not None:
            country = tweet.place.country
        else:
            country = None
        relevant = df_relevant_users[df_relevant_users['user_id'] == user_id]['relevant_nivel'].values[0] if user_id in df_relevant_users['user_id'].values else None
        

        tweet_df = pd.DataFrame({'tweet_id': [tweet_id],
                                    'timestamp': [timestamp], 
                                    'user_id': [user_id], 
                                    'screen_name': [screen_name], 
                                    'full_name': [full_name], 
                                    'text': [text], 
                                    'hashtags': [hashtags], 
                                    'mentions': [mentions], 
                                    'retweet_count': [retweet_count],
                                    'retweet_user' : [retweet_user], 
                                    'favorite_count': [favorite_count],
                                    'country':[country],
                                    'relevant': [relevant]
                                    })
        # Concatenar el DataFrame del tweet con el DataFrame principal
        tweets_df = pd.concat([tweets_df, tweet_df], ignore_index=True)

In [91]:
# Ordena los tweets por fecha y resetea el índice
tweets_df = tweets_df.sort_values(by='timestamp').reset_index(drop=True)

In [92]:
# Visualiza el DataFrame resultante
tweets_df_ = tweets_df[tweets_df['retweet_count']>0] # mascara que filtra los RT
tweets_df.head(30)


,tweet_id,timestamp,user_id,screen_name,full_name,text,hashtags,mentions,retweet_count,retweet_user,favorite_count,tweet_sentiment
0,1657526740598792194,2023-05-13 23:22:24+00:00,1010838965702856704,deyonte_btc,Deyonté ⚡️,@JanuaryxEC 💯💯💯,[],[JanuaryxEC],0,[],0,NaN
1,1657526750203768832,2023-05-13 23:22:27+00:00,938861584914505728,recovery_btc,bitcoin recovery service,@CryptoCanonist Thanks again,[],[CryptoCanonist],0,[],0,NaN
2,1657526759439609856,2023-05-13 23:22:29+00:00,1256538305174179840,ManhL0c,Tatsuya,"3/ Yes, staking rewards are automatically comp...","[CompoundRewards, StakeAndEarn, Crypto, Stakin...",[],0,[],0,NaN
3,1657526757635809282,2023-05-13 23:22:29+00:00,1572065656899616768,GriefZHODL,40/400 GriefZ,@The_BTC_Patriot Once you got older you’d abso...,[],[The_BTC_Patriot],0,[],0,NaN
4,1657526760987033601,2023-05-13 23:22:29+00:00,1470523446249537537,peakbtc,3PA,no volatitility feels like \n\n$BTC $ETH https...,[],[],0,[],0,NaN
5,1657526769665097728,2023-05-13 23:22:31+00:00,368429535,dkrazey,derekL,@APompliano $goog $msft will be better than $BTC,[],[APompliano],0,[],0,NaN
6,1657526770684383234,2023-05-13 23:22:32+00:00,17501714,E_Wiz_abeth,Elizabeth J.,👇 I just been airdropped $YODA Thank you guys ...,[],[],0,[],0,NaN
7,1657526771326291969,2023-05-13 23:22:32+00:00,1358272480851333120,CucuBeni,Ben Tjutju Avive Citizen,#Avive - Bukti Protokol Jaringan\n\nSaya telah...,"[Avive, Avive, Avive, BTC]",[],0,[],0,NaN
8,1657526775176470529,2023-05-13 23:22:33+00:00,1439743773039333378,ilkerkuzucuu,İLKER,#pepe #btc ytd . https://t.co/hCl4G8WssF,"[pepe, btc]",[],1,[ChhrazySar79198],1,NaN
9,1657526810584776705,2023-05-13 23:22:41+00:00,36408246,Sgrhomika22,TAMIKA,👇I moved all my crypto into $YODA this one wil...,[],[],0,[],0,NaN


In [93]:
tweets_df.shape
tweets_df.to_csv('tweets.csv', index=False)